In [1]:
# ============================================================
# Extra: integração com WWF Ecoregions (biomas + reinos)
# ============================================================

!pip install geopandas requests shapely --quiet
import geopandas as gpd, pandas as pd, requests, zipfile, os, io
import time

# URL alternativa (mirror confiável do shapefile WWF ecoregions v2)
WWF_URL = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip" # Using a different URL for demonstration, replace with a reliable WWF source if available.

# 📥 Download seguro do ZIP com retry
print("Baixando shapefile das ecorregiões WWF...")
max_retries = 3
for attempt in range(max_retries):
    try:
        r = requests.get(WWF_URL, stream=True, timeout=10) # Added timeout
        r.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        break # Success, exit the loop
    except requests.exceptions.RequestException as e:
        print(f"Tentativa {attempt + 1} falhou: {e}")
        if attempt < max_retries - 1:
            time.sleep(5) # Wait before retrying
        else:
            raise RuntimeError("Erro no download das ecorregiões WWF após várias tentativas.") from e


with open("wwf_ecoregions.zip", "wb") as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)

# 📦 Extração
os.makedirs("wwf_ecoregions", exist_ok=True)
with zipfile.ZipFile("wwf_ecoregions.zip", "r") as z:
    z.extractall("wwf_ecoregions")

# 🔍 Localiza o shapefile (pode estar em subpastas)
import glob
shp_path = [p for p in glob.glob("wwf_ecoregions/**/*.shp", recursive=True)
            if "Ecoregions2017" in os.path.basename(p)][0]

# 📚 Leitura do shapefile WWF
ecoreg = gpd.read_file(shp_path)
ecoreg = ecore.to_crs("EPSG:4326")[["ECO_NAME","BIOME_NAME","REALM","geometry"]]
print(f"{len(ecoreg)} ecorregiões carregadas.")

# ✅ Garante que você tenha world_wgs (países em lat/lon)
assert 'world_wgs' in locals(), "world_wgs não encontrado — rode antes a parte geográfica do notebook."

# 🧬 Interseção espacial: qual ecorregião cobre cada país
# (usando centróide representativo de cada país)
world_pts = gpd.GeoDataFrame(world_wgs[[NAME_WORLD,"rep_pt"]].copy(),
                             geometry=world_wgs["rep_pt"], crs="EPSG:4326")
joined = gpd.sjoin(world_pts, ecoreg, how="left", predicate="within")

# 🔗 Merge de volta no GeoDataFrame principal
world_wgs = world_wgs.merge(
    joined[[NAME_WORLD,"ECO_NAME","BIOME_NAME","REALM"]],
    on=NAME_WORLD, how="left"
)

# ✨ Colunas adicionadas:
# ECO_NAME  -> nome detalhado da ecorregião WWF
# BIOME_NAME -> bioma WWF (ex: Tropical Moist Broadleaf Forests)
# REALM -> reino biogeográfico (Neotropical, Palearctic etc.)
print("✅ Ecoregions WWF adicionadas com sucesso.")
world_wgs[["NAME","ECO_NAME","BIOME_NAME","REALM"]].head()

Baixando shapefile das ecorregiões WWF...


RuntimeError: Erro no download das ecorregiões WWF.